<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Partial-updates" data-toc-modified-id="Partial-updates-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Partial updates</a></span></li></ul></div>

In [86]:
import os
import requests
import pprint
import pandas as pd
import pandas_gbq
import numpy as np
import re
import itertools

from time import sleep
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from datetime import datetime
from requests_html import HTML , HTMLSession

In [206]:
HOME = "https://www.hemnet.se"
url=HOME + "/sitemap"
def find_el_by_cl(r_html, el='li', cl=('sitemap__list-item',)):
        els=r_html.find(el)
        return [el for el in els if el.attrs.get('class')==cl]
        

def get_lists(url):
    r=requests.get(url)
    r_html=HTML(html=r.text)
    els=find_el_by_cl(r_html,  el='li', cl=('sitemap__list-item',) )
    els=[el.find('li')[0].find('a')[0].attrs.get('href') for el in els]
    return  [el for el in els if el.startswith('/sitemap/plats/')]
lans=get_lists(url)

In [262]:
#find_el_by_cl(r_html, el='h1', cl='sitemap__heading')
city=r_html.find('h1')[0].text.split(' ')[2]

'Karlshamns'

In [209]:

lans_urls = [HOME+el for el in lans]

In [222]:
def get_urls(urls):
    return [get_lists(url) for url in urls]
komms_raw=get_urls(lans_urls)

In [226]:
komms=list(itertools.chain(*komms_raw))
komms_urls=[HOME+el for el in komms]
komms_urls

['https://www.hemnet.se/sitemap/plats/blekinge-lan/karlshamns-kommun',
 'https://www.hemnet.se/sitemap/plats/blekinge-lan/karlskrona-kommun',
 'https://www.hemnet.se/sitemap/plats/blekinge-lan/olofstroms-kommun',
 'https://www.hemnet.se/sitemap/plats/blekinge-lan/ronneby-kommun',
 'https://www.hemnet.se/sitemap/plats/blekinge-lan/solvesborgs-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/avesta-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/borlange-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/falu-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/gagnefs-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/hedemora-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/leksands-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/ludvika-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/malung-salens-kommun',
 'https://www.hemnet.se/sitemap/plats/dalarnas-lan/mora-kommun',
 'https://www.hemnet.se/sitemap/p

In [212]:
# communs=[HOME+'/salda/'+url.split('/')[-1] for url in urls_list]
# communs

In [84]:
pd.DataFrame({'communs':communs}).to_pickle('communs')

In [302]:
def get_districts(komms_url):
    sleep(1)
    r_html=HTML(html=requests.get(komms_url).text)
    city=r_html.find('h1')[0].text.split(' ')[2]
    els =[re.sub("[\(\[].*?[\)\]]", "", el.text.replace('+', '-')) for el in find_el_by_cl(r_html, el='li', cl=('sitemap__list-item',))]
    districts=np.unique(list(itertools.chain(*[[x.strip() for x in d.split('-') if x !=''] for d in els])))
    districts = pd.DataFrame({'district':districts})
    districts['city']=city
    return districts

districts_raw=[get_districts(url) for url in komms_urls]


In [303]:
dim_districts=pd.concat(districts_raw)
dim_districts=pd.concat([dim_districts, 
 pd.DataFrame({'district':['Tycho Hedéns väg', 'Dag Hammarskjölds Väg'], 'city': ['Uppsala', 'Uppsala']})])

In [308]:
pandas_gbq.to_gbq(dim_districts, destination_table='crime_statistics.dim_districts_hemnet', 
                  project_id= os.environ['GCP_PROJECID'], if_exists='replace')

1it [00:02,  2.93s/it]


In [271]:
q_url=f"""
https://nominatim.openstreetmap.org/search/405 Säter Sweden?format=json&limit=1
"""
requests.get(q_url).text

'[{"place_id":308864,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"node","osm_id":124274960,"boundingbox":["60.3069718","60.3869718","15.7086886","15.7886886"],"lat":"60.3469718","lon":"15.7486886","display_name":"Säter, Säters kommun, Dalarnas län, 783 30, Sverige","class":"place","type":"town","importance":0.41000000000000003,"icon":"https://nominatim.openstreetmap.org/ui/mapicons//poi_place_town.p.20.png"}]'

In [281]:
import time
tic=time.time()
df = pandas_gbq.read_gbq("""SELECT * FROM `my-user-project-287504.crime_statistics.cities_refined` 
                    """, project_id=os.environ['GCP_PROJECID'])
toc=time.time()
print(toc-tic)


Downloading:   0%|          | 0/6350 [00:33<?, ?rows/s]

Downloading: 100%|██████████| 6350/6350 [00:01<00:00, 4920.35rows/s]

2.405704975128174


In [282]:
pandas_gbq.to_gbq(dim_districts, destination_table='crime_statistics.dim_districts_hemnet', 
                  project_id= os.environ['GCP_PROJECID'], if_exists='replace')

1it [00:04,  4.77s/it]


In [ ]:
### Partial updates

#ToDo: 
# city_refined
# 1. pull from bq
# extract location details again
# get osm coordinates
# upload to bq


# city_refined_en
# 1. pull from bq
# extract location details again
# get osm coordinates
# upload to bq
